# Import modules

In [1]:
from __future__ import division

import os; os.chdir(os.path.join('..', '..'))
print os.getcwd()

/home/eharkin/projects/5HTGIF-dev


In [2]:
import warnings
import re

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import seaborn as sns
import numpy as np

from grr.Simulation import GIFnet_Simulation
from grr.Tools import PSTH

# Load data

Put `GIFnet_Simulation` objects into a DataFrame with columns to describe experimental conditions.

In [3]:
DATA_PATH = os.path.join('data', 'simulations', 'GIF_network')

sim_dict = {'Mod type': [], 'Condition': [], 'Rep': [], 'Model': []}

for directory in ['subsample_base', 'subsample_noIA', 'subsample_fixedIA']:
    for fname in os.listdir(os.path.join(DATA_PATH, directory)):
        if 'hdf5' in fname.lower() or 'h5' in fname.lower():
        
            if 'base' in fname:
                sim_dict['Mod type'].append('Base')
            elif 'noIA' in fname:
                sim_dict['Mod type'].append('No IA')
            elif 'fixedIA' in fname:
                sim_dict['Mod type'].append('Fixed IA')
            else:
                sim_dict['Mod type'].append('Unknown')

            if 'noGABA' in fname:
                sim_dict['Condition'].append('No GABA')
            elif 'GABA' in fname:
                sim_dict['Condition'].append('Control')
            else:
                sim_dict['Condition'].append('Unknown')

            sim_dict['Rep'].append(
                int(re.search(r'rep([0-9]+)', fname).groups()[-1])
            )

            sim_dict['Model'].append(GIFnet_Simulation(os.path.join(DATA_PATH, directory, fname), mode = 'r'))

sim_df = pd.DataFrame(sim_dict).sort_values(['Mod type', 'Condition', 'Rep'])
sim_df.reset_index(inplace=True, drop=True)
sim_df.head()

,Condition,Mod type,Model,Rep
0,Control,Base,"[connectivity_matrix, gaba, ser]",0
1,Control,Base,"[connectivity_matrix, gaba, ser]",1
2,Control,Base,"[connectivity_matrix, gaba, ser]",2
3,Control,Base,"[connectivity_matrix, gaba, ser]",3
4,Control,Base,"[connectivity_matrix, gaba, ser]",4


# Analysis
## Extract PSTH

In [4]:
sim_df['5HT PSTH'] = None
sim_df['5HT PSTH'] = sim_df['5HT PSTH'].astype(object)
sim_df['GABA PSTH'] = None
sim_df['GABA PSTH'] = sim_df['GABA PSTH'].astype(object)

In [5]:
PSTH_width = 25.

for i in range(sim_df.shape[0]):
    print('Computing PSTH {:.1f}%'.format(100. * i / sim_df.shape[0]))
    
    # Get PSTH from 5HT population.
    sim_df.at[i, '5HT PSTH'] = np.empty((
        sim_df.loc[i, 'Model'].get_no_sweeps(), 
        sim_df.loc[i, 'Model'].get_no_timesteps()
    ))
    for sweep_no in range(sim_df.loc[i, 'Model'].get_no_sweeps()):
        sim_df.loc[i, '5HT PSTH'][sweep_no, :] = PSTH(
            sim_df.loc[i, 'Model'].ser_spktrains[sweep_no, ...].sum(axis = 0),
            PSTH_width,
            sim_df.loc[i, 'Model'].get_no_ser_neurons()
        )
    
    # Get PSTHs from GABA population.
    if sim_df.loc[i, 'Condition'] == 'No GABA':
        sim_df.loc[i, 'GABA PSTH'] = None
        continue
    else:
        sim_df.at[i, 'GABA PSTH'] = np.empty((
            sim_df.loc[i, 'Model'].get_no_sweeps(), 
            sim_df.loc[i, 'Model'].get_no_timesteps()
        ))
        for sweep_no in range(sim_df.loc[i, 'Model'].get_no_sweeps()):
            sim_df.loc[i, 'GABA PSTH'][sweep_no, :] = PSTH(
                sim_df.loc[i, 'Model'].gaba_spktrains[sweep_no, ...].sum(axis = 0),
                PSTH_width,
                sim_df.loc[i, 'Model'].get_no_gaba_neurons()
            )

Computing PSTH 0.0%
Computing PSTH 1.7%
Computing PSTH 3.3%
Computing PSTH 5.0%
Computing PSTH 6.7%
Computing PSTH 8.3%
Computing PSTH 10.0%
Computing PSTH 11.7%
Computing PSTH 13.3%
Computing PSTH 15.0%
Computing PSTH 16.7%
Computing PSTH 18.3%
Computing PSTH 20.0%
Computing PSTH 21.7%
Computing PSTH 23.3%
Computing PSTH 25.0%
Computing PSTH 26.7%
Computing PSTH 28.3%
Computing PSTH 30.0%
Computing PSTH 31.7%
Computing PSTH 33.3%
Computing PSTH 35.0%
Computing PSTH 36.7%
Computing PSTH 38.3%
Computing PSTH 40.0%
Computing PSTH 41.7%
Computing PSTH 43.3%
Computing PSTH 45.0%
Computing PSTH 46.7%
Computing PSTH 48.3%
Computing PSTH 50.0%
Computing PSTH 51.7%
Computing PSTH 53.3%
Computing PSTH 55.0%
Computing PSTH 56.7%
Computing PSTH 58.3%
Computing PSTH 60.0%
Computing PSTH 61.7%
Computing PSTH 63.3%
Computing PSTH 65.0%
Computing PSTH 66.7%
Computing PSTH 68.3%
Computing PSTH 70.0%
Computing PSTH 71.7%
Computing PSTH 73.3%
Computing PSTH 75.0%
Computing PSTH 76.7%
Computing PSTH 78.3

In [ ]:
psth_dict = {'Mod type': [], 'Condition': [], '5HT PSTH': []}
for modtype in sim_df['Mod type'].unique():
    for cond in sim_df['Condition'].unique():
        psth_dict['Mod type'].append(modtype)
        psth_dict['Condition'].append(cond)
        psth_dict['5HT PSTH'].append(np.array(
            sim_df.loc[(sim_df['Condition'] == cond) & (sim_df['Mod type'] == modtype), '5HT PSTH'].tolist()
        ))
psth_df = pd.DataFrame(psth_dict)
del psth_dict

In [ ]:
psth_df.head()

In [ ]:
psth_df['5HT PSTH'][0].shape

Each 5HT PSTH has dimensionality `[repeats, sweeps, time]`.

In [ ]:
t_vec = np.arange(0, sim_df.loc[0, 'Model'].attrs['T'], sim_df.loc[0, 'Model'].attrs['dt'])
for modtype in psth_df['Mod type'].unique():
    fig, ax = plt.subplots(1, len(sim_df['Condition'].unique()), sharey=True, figsize=(8, 4))
    ax[0].set_ylabel('Population PSTH (Hz)')
    
    plt.suptitle(modtype)
    for i, cond in enumerate(psth_df['Condition'].unique()):
        ax[i].set_title(cond)
        ax[i].plot(
            t_vec,
            psth_df.loc[
                (psth_df['Condition'] == cond) & (psth_df['Mod type'] == modtype), '5HT PSTH'
            ].item().mean(axis=0).T
        )
        ax[i].set_xlim(800, 2000)
        ax[i].set_xlabel('Time (ms)')
    
    plt.tight_layout()
    plt.show()

In [ ]:
time_slice = slice(10000, 12000)

fig, ax = plt.subplots(1, 1)

ax.set_title('5HT Population f/I curves')
ax.plot(
    psth_df.loc[
        (psth_df['Condition'] == 'Control') & (psth_df['Mod type'] == 'Base'), '5HT PSTH'
    ].item()[..., time_slice].max(axis=2).mean(axis=0),
    'k-',
    label='Base CTRL'
)
ax.plot(
    psth_df.loc[
        (psth_df['Condition'] == 'No GABA') & (psth_df['Mod type'] == 'Base'), '5HT PSTH'
    ].item()[..., time_slice].max(axis=2).mean(axis=0),
    'k--',
    label='Base no GABA'
)

ax.plot(
    psth_df.loc[
        (psth_df['Condition'] == 'Control') & (psth_df['Mod type'] == 'No IA'), '5HT PSTH'
    ].item()[..., time_slice].max(axis=2).mean(axis=0),
    '-', color='gray', alpha=0.5,
    label='IA KO CTRL'
)
ax.plot(
    psth_df.loc[
        (psth_df['Condition'] == 'No GABA') & (psth_df['Mod type'] == 'No IA'), '5HT PSTH'
    ].item()[..., time_slice].max(axis=2).mean(axis=0),
    '--', color='gray', alpha=0.5,
    label='IA KO no GABA'
)

ax.plot(
    psth_df.loc[
        (psth_df['Condition'] == 'Control') & (psth_df['Mod type'] == 'Fixed IA'), '5HT PSTH'
    ].item()[..., time_slice].max(axis=2).mean(axis=0),
    'b-', alpha=0.5,
    label='Fixed IA CTRL'
)
ax.plot(
    psth_df.loc[
        (psth_df['Condition'] == 'No GABA') & (psth_df['Mod type'] == 'Fixed IA'), '5HT PSTH'
    ].item()[..., time_slice].max(axis=2).mean(axis=0),
    'b--', alpha=0.5,
    label='Fixed IA no GABA'
)
ax.set_ylabel('5HT population PSTH peak (Hz)')
ax.set_xlabel('Sweep no.')

plt.show()

In [ ]:
arrs = []
for condition in psth_df['Condition'].unique():
    for modtype in psth_df['Mod type'].unique():
        arr = psth_df.loc[
            (psth_df['Condition'] == condition) & (psth_df['Mod type'] == modtype), '5HT PSTH'
        ].item()[..., time_slice].max(axis=2).astype(object)
        arrs.append(np.concatenate(
            [np.array([condition for i in range(arr.size)])[:, np.newaxis],  # Condition column
            np.array([modtype for i in range(arr.size)])[:, np.newaxis],    # Model type column
            np.array([i % arr.shape[0] for i in range(arr.size)])[:, np.newaxis],  # Sweep no. column
            arr.flatten()[:, np.newaxis]],
            axis=1
        ))

arrs = np.concatenate(arrs, axis=0)
peak_df = pd.DataFrame(arrs, columns=['Condition', 'Mod type', 'Sweep', 'PSTH peak']).infer_objects()
peak_df.head()

In [ ]:
sns.lmplot('Sweep', 'PSTH peak', hue='Condition', data = peak_df.loc[peak_df['Mod type'] == 'Fixed IA', :])

In [ ]:
sns.lmplot('Sweep', 'PSTH peak', hue='Condition', data = peak_df.loc[peak_df['Mod type'] == 'No IA', :])

In [ ]:
peak_df.to_csv('PSTH_peak.csv')

Analyse dataframe in r

## Plot I/O curves

In [ ]:
def keyfilt_generator(dict_, prefix=None, model=None, baseline=None, gaba=None, rep=None):
    """Restrict _ delimited keys to specified values, or any for None."""
    for key in dict_:
        
        splitkey = key.split('_')
        assert len(splitkey) == 5, 'Expected {} to have 5 `_`-delimited entries, not {}'.format(key, len(splitkey))
        
        rejectkey = False
        for keyitem, filt in zip(splitkey, [prefix, model, baseline, gaba, rep]):
            if filt is None or keyitem in filt:
                continue
            else:
                rejectkey = True
                break
                
        if rejectkey:
            continue
        else:
            yield dict_[key]
        

In [ ]:
def condname(prefix, model, baseline, gaba, rep):
    """Throwaway function to build keys in a consistent way.
    """
    return '_'.join([prefix, model, baseline, gaba, rep])

In [ ]:
time_slice = slice(9500, 11000)

plt.figure(figsize = (10, 8))

axes = {}
spec_outer = gs.GridSpec(3, 2)

# Column 1: base model
axes['base_ser_psth'] = plt.subplot(spec_outer[0, 0])
axes['base_ser_psth'].set_title('Base')
for dat in keyfilt_generator(ser_PSTHs, ['subsample'], ['base'], ['m'], ['g'], None):
    axes['base_ser_psth'].plot(dat[::2, time_slice].T, 'b--')
    break  # Hacky way to plot only one example replicate.
for dat in keyfilt_generator(ser_PSTHs, ['subsample'], ['base'], ['m'], ['ng'], None):
    axes['base_ser_psth'].plot(dat[::2, time_slice].T, 'k-')
    break
axes['base_ser_psth'].set_xticks([])
axes['base_ser_psth'].set_ylabel('5HT pop. firing rate\n(Hz/neuron)')

axes['base_gaba_psth'] = plt.subplot(spec_outer[1, 0])
for dat in keyfilt_generator(gaba_PSTHs, ['subsample'], ['base'], ['m'], ['g'], None):
    axes['base_gaba_psth'].plot(dat[::2, time_slice].T, 'b-')
    break
axes['base_gaba_psth'].set_ylabel('GABA pop. firing rate\n(Hz/neuron)')
axes['base_gaba_psth'].set_xlabel('Time (timesteps)')

axes['base_peak'] = plt.subplot(spec_outer[2, 0])
ind = 0
for dat in keyfilt_generator(ser_PSTHs, ['subsample'], ['base'], ['m'], ['ng'], None):
    if ind == 0:
        axes['base_peak'].plot(dat[:, time_slice].max(axis=1), 'k-', label='No GABA')
    else:
        axes['base_peak'].plot(dat[:, time_slice].max(axis=1), 'k-')
    ind += 1
ind = 0
for dat in keyfilt_generator(ser_PSTHs, ['subsample'], ['base'], ['m'], ['g'], None):
    if ind == 0:
        axes['base_peak'].plot(dat[:, time_slice].max(axis=1), 'b-', label='Control')
    else:
        axes['base_peak'].plot(dat[:, time_slice].max(axis=1), 'b-')
    ind += 1
axes['base_peak'].set_ylabel('Peak 5HT pop. firing rate\n(Hz/neuron)')
axes['base_peak'].set_xlabel('Stim no.')
axes['base_peak'].legend()

# Column 2: Model with IA knocked out
axes['noIA_ser_psth'] = plt.subplot(spec_outer[0, 1])
axes['noIA_ser_psth'].set_title('IA KO')
for dat in keyfilt_generator(ser_PSTHs, ['subsample'], ['noIA'], ['m'], ['g'], None):
    axes['noIA_ser_psth'].plot(dat[::2, time_slice].T, 'b--')
    break  # Hacky way to plot only one example replicate.
for dat in keyfilt_generator(ser_PSTHs, ['subsample'], ['noIA'], ['m'], ['ng'], None):
    axes['noIA_ser_psth'].plot(dat[::2, time_slice].T, 'k-')
    break
axes['noIA_ser_psth'].set_xticks([])
axes['noIA_ser_psth'].set_ylabel('5HT pop. firing rate\n(Hz/neuron)')

axes['noIA_gaba_psth'] = plt.subplot(spec_outer[1, 1])
for dat in keyfilt_generator(gaba_PSTHs, ['subsample'], ['noIA'], ['m'], ['g'], None):
    axes['noIA_gaba_psth'].plot(dat[::2, time_slice].T, 'b-')
    break
axes['noIA_gaba_psth'].set_ylabel('GABA pop. firing rate\n(Hz/neuron)')
axes['noIA_gaba_psth'].set_xlabel('Time (timesteps)')

axes['noIA_peak'] = plt.subplot(spec_outer[2, 1])
ind = 0
for dat in keyfilt_generator(ser_PSTHs, ['subsample'], ['noIA'], ['m'], ['ng'], None):
    if ind == 0:
        axes['noIA_peak'].plot(dat[:, time_slice].max(axis=1), 'k-', label='No GABA')
    else:
        axes['noIA_peak'].plot(dat[:, time_slice].max(axis=1), 'k-')
    ind += 1
ind = 0
for dat in keyfilt_generator(ser_PSTHs, ['subsample'], ['noIA'], ['m'], ['g'], None):
    if ind == 0:
        axes['noIA_peak'].plot(dat[:, time_slice].max(axis=1), 'b-', label='Control')
    else:
        axes['noIA_peak'].plot(dat[:, time_slice].max(axis=1), 'b-')
    ind += 1
axes['noIA_peak'].set_ylabel('Peak 5HT pop. firing rate\n(Hz/neuron)')
axes['noIA_peak'].set_xlabel('Stim no.')
axes['noIA_peak'].legend()

plt.tight_layout()
plt.show()

In [ ]:
sweep_no = 8
spktimes = {'base': {'g': [], 'ng': []}, 'noIA': {'g': [], 'ng': []}}
for mod_type in spktimes:
    for gaba_cond in spktimes[mod_type]:
        i = 0
        for dat in keyfilt_generator(simfiles, ['subsample'], [mod_type], ['m'], [gaba_cond], None):
            print 'Getting spktimes {} {} {}'.format(mod_type, gaba_cond, i)
            spktimes_tmp = dat.get_ser_spktimes()
            spktimes_sw_tmp = np.concatenate(spktimes_tmp[sweep_no])
            spktimes_sw_tmp.sort()
            spktimes[mod_type][gaba_cond].append(spktimes_sw_tmp)
            i += 1

del mod_type, gaba_cond, dat, spktimes_tmp, spktimes_sw_tmp, i

In [ ]:
start_time = 1000
stop_time = 1400

plt.figure(figsize=(8, 8))

plt.subplot(221)
plt.title('Base')
for spktimes_tmp in spktimes['base']['g']:
    mask = (spktimes_tmp >= start_time) & (spktimes_tmp < stop_time)
    sns.kdeplot(spktimes_tmp[mask] - start_time, ax=plt.gca(), color='b')
for spktimes_tmp in spktimes['base']['ng']:
    mask = (spktimes_tmp >= start_time) & (spktimes_tmp < stop_time)
    sns.kdeplot(spktimes_tmp[mask] - start_time, ax=plt.gca(), color='k')
plt.yticks([])
plt.xscale('log')
plt.xlim(1, stop_time - start_time)
plt.ylabel('Density')

plt.subplot(223)
for i, spktimes_tmp in enumerate(spktimes['base']['g']):
    mask = (spktimes_tmp >= start_time) & (spktimes_tmp < stop_time)
    if i == 0:
        plt.semilogx(
            spktimes_tmp[mask] - start_time, 
            np.linspace(1/len(spktimes_tmp), 1., len(spktimes_tmp))[mask], 
            'b-', label='Control'
        )
    else:
        plt.semilogx(
            spktimes_tmp[mask] - start_time, 
            np.linspace(1/len(spktimes_tmp), 1., len(spktimes_tmp))[mask], 
            'b-'
        )
for i, spktimes_tmp in enumerate(spktimes['base']['ng']):
    mask = (spktimes_tmp >= start_time) & (spktimes_tmp < stop_time)
    if i == 0:
        plt.semilogx(
            spktimes_tmp[mask] - start_time, 
            np.linspace(1/len(spktimes_tmp), 1., len(spktimes_tmp))[mask], 
            'k-', label='No GABA'
        )
    else:
        plt.semilogx(
            spktimes_tmp[mask] - start_time, 
            np.linspace(1/len(spktimes_tmp), 1., len(spktimes_tmp))[mask], 
            'k-'
        )
plt.ylabel('Cumulative fraction')
plt.xlabel('Time from stim onset')
plt.legend()

plt.subplot(222)
plt.title('IA KO')
for spktimes_tmp in spktimes['noIA']['g']:
    mask = (spktimes_tmp >= start_time) & (spktimes_tmp < stop_time)
    sns.kdeplot(spktimes_tmp[mask] - start_time, ax=plt.gca(), color='b')
for spktimes_tmp in spktimes['noIA']['ng']:
    mask = (spktimes_tmp >= start_time) & (spktimes_tmp < stop_time)
    sns.kdeplot(spktimes_tmp[mask] - start_time, ax=plt.gca(), color='k')
plt.yticks([])
plt.xscale('log')
plt.xlim(1, stop_time - start_time)

plt.subplot(224)
for i, spktimes_tmp in enumerate(spktimes['noIA']['g']):
    mask = (spktimes_tmp >= start_time) & (spktimes_tmp < stop_time)
    if i == 0:
        plt.semilogx(
            spktimes_tmp[mask] - start_time, 
            np.linspace(1/len(spktimes_tmp), 1., len(spktimes_tmp))[mask], 
            'b-', label='Control'
        )
    else:
        plt.semilogx(
            spktimes_tmp[mask] - start_time, 
            np.linspace(1/len(spktimes_tmp), 1., len(spktimes_tmp))[mask], 
            'b-'
        )
for i, spktimes_tmp in enumerate(spktimes['noIA']['ng']):
    mask = (spktimes_tmp >= start_time) & (spktimes_tmp < stop_time)
    if i == 0:
        plt.semilogx(
            spktimes_tmp[mask] - start_time, 
            np.linspace(1/len(spktimes_tmp), 1., len(spktimes_tmp))[mask], 
            'k-', label='No GABA'
        )
    else:
        plt.semilogx(
            spktimes_tmp[mask] - start_time, 
            np.linspace(1/len(spktimes_tmp), 1., len(spktimes_tmp))[mask], 
            'k-'
        )
plt.xlabel('Time from stim onset')
plt.legend()

plt.show()

del spktimes_tmp, i

Top row: KDE plots of spike time PDF for each replicate. Bottom row: cumulative spike time distribution plots for each replicate.

In [ ]:
def masked_kde(spktimes, start_time, stop_time, **pltargs):
    spktimes_tmp = np.concatenate(spktimes)
    mask = (spktimes_tmp >= start_time) & (spktimes_tmp < stop_time)
    sns.kdeplot(spktimes_tmp[mask] - start_time, **pltargs)


def masked_cumdist(spktimes, start_time, stop_time, **pltargs):
    spktimes_tmp = np.concatenate(spktimes).copy()
    spktimes_tmp.sort()
    mask = (spktimes_tmp >= start_time) & (spktimes_tmp < stop_time)
    plt.plot(
        spktimes_tmp[mask] - start_time, 
        np.linspace(1/len(spktimes_tmp), 1., len(spktimes_tmp))[mask], 
        **pltargs
    )


plt.figure(figsize=(10, 6))

plt.subplot(121)
masked_kde(spktimes['base']['g'], start_time, stop_time, color='b', label='Base model ctrl')
masked_kde(spktimes['base']['ng'], start_time, stop_time, color='k', label='Base model no GABA')
masked_kde(spktimes['noIA']['g'], start_time, stop_time, color='b', linestyle='--', label='IA KO ctrl')
masked_kde(spktimes['noIA']['ng'], start_time, stop_time, color='k', linestyle='--', label='IA KO no GABA')
plt.xscale('log')
plt.xlim(1, stop_time - start_time)
plt.ylim(0, plt.ylim()[1] * 1.2)
plt.yticks([])
plt.ylabel('Density')
plt.xlabel('Time from stim onset')
plt.legend()

plt.subplot(122)
masked_cumdist(spktimes['base']['g'], start_time, stop_time, color='b', label='Base model ctrl')
masked_cumdist(spktimes['base']['ng'], start_time, stop_time, color='k', label='Base model no GABA')
masked_cumdist(spktimes['noIA']['g'], start_time, stop_time, color='b', linestyle='--', label='IA KO ctrl')
masked_cumdist(spktimes['noIA']['ng'], start_time, stop_time, color='k', linestyle='--', label='IA KO no GABA')
plt.xscale('log')
plt.xlim(1, stop_time - start_time)
plt.ylabel('Cumulative fraction')
plt.xlabel('Time from stim onset')
plt.legend()

plt.show()

# Close all files

In [ ]:
for fname in simfiles.keys():
    simfiles[fname].close()